In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r drive/"My Drive"/"Dutch Tweets" .

cp: cannot open 'drive/My Drive/Dutch Tweets/allDataList_chunk9.gsheet' for reading: Operation not supported
cp: cannot open 'drive/My Drive/Dutch Tweets/allDataList_chunk6.gsheet' for reading: Operation not supported


In [ ]:
# Read location chunks 
import pandas as pd 

idx = 6

allLocation_chunk = pd.read_csv("Dutch Tweets/allLocation_chunk"+str(idx)+".csv")
allLocation_chunk.shape
allLocation_chunk.head()

,Unnamed: 0,location
0,9171,"Elim, PA"
1,9172,@VlkEke
2,9173,Staatsbosbeheer Noord Limburg
3,9174,Zennewijnen
4,9175,IJsselmuiden


In [ ]:
import geopy.geocoders
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

locator = Nominatim(user_agent="myGeocoder")

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)


In [ ]:
# 2- - create location column
allLocation_chunk['point_info'] = allLocation_chunk['location'].apply(geocode)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('MEXICO-III • MEXICO',), **{}).
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/geopy/geocoders/base.py", line 344, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "/usr/lib/python3.6/urllib/request.py", line 526, in open
    response = self._open(req, data)
  File "/usr/lib/python3.6/urllib/request.py", line 544, in _open
    '_open', req)
  File "/usr/lib/python3.6/urllib/request.py", line 504, in _call_chain
    result = func(*args)
  File "/usr/lib/python3.6/urllib/request.py", line 1368, in https_open
    context=self._context, check_hostname=self._check_hostname)
  File "/usr/lib/python3.6/urllib/request.py", line 1328, in do_open
    r = h.getresponse()
  File "/usr/lib/python3.6/http/client.py", line 1356, in getresponse
    response.begin()
  File "/usr/lib/python3.6/http/client.py", line 307, in begin
    version, status, reason = self._read_status

In [ ]:
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
allLocation_chunk['point'] = allLocation_chunk['point_info'].apply(lambda loc: tuple(loc.point) if loc else None)

allLocation_chunk.head()

,Unnamed: 0,location,point_info,point
0,9171,"Elim, PA","(Elim, Upper Yoder Township, Cambria County, P...","(40.2987335, -78.94353268168663, 0.0)"
1,9172,@VlkEke,None,None
2,9173,Staatsbosbeheer Noord Limburg,None,None
3,9174,Zennewijnen,"(Zennewijnen, Tiel, Gelderland, Nederland, (51...","(51.8587463, 5.4118529, 0.0)"
4,9175,IJsselmuiden,"(IJsselmuiden, Kampen, Overijssel, Nederland, ...","(52.56396385, 5.940171304145334, 0.0)"


In [ ]:
# 4 - split point column into latitude, longitude and altitude columns
allLocation_chunk[['latitude', 'longitude', 'altitude']] = pd.DataFrame(allLocation_chunk['point'].tolist(), index=allLocation_chunk.index)

allLocation_chunk.head(15)

,Unnamed: 0,location,point_info,point,latitude,longitude,altitude
0,9171,"Elim, PA","(Elim, Upper Yoder Township, Cambria County, P...","(40.2987335, -78.94353268168663, 0.0)",40.298733,-78.943533,0.0
1,9172,@VlkEke,None,None,NaN,NaN,NaN
2,9173,Staatsbosbeheer Noord Limburg,None,None,NaN,NaN,NaN
3,9174,Zennewijnen,"(Zennewijnen, Tiel, Gelderland, Nederland, (51...","(51.8587463, 5.4118529, 0.0)",51.858746,5.411853,0.0
4,9175,IJsselmuiden,"(IJsselmuiden, Kampen, Overijssel, Nederland, ...","(52.56396385, 5.940171304145334, 0.0)",52.563964,5.940171,0.0
5,9176,"Lima, Perú","(Lima, Peru, (-12.0621065, -77.0365256))","(-12.0621065, -77.0365256, 0.0)",-12.062107,-77.036526,0.0
6,9177,Knoal,"(Knoal Hutte, Obergailer Talweg, Obergail, Les...","(46.6517372, 12.790946938592818, 0.0)",46.651737,12.790947,0.0
7,9178,Haarlem -A'dam M'tricht Aalt'n,None,None,NaN,NaN,NaN
8,9179,"Macragge, Ultramar sector",None,None,NaN,NaN,NaN
9,9180,Zeeland/Sud de France,None,None,NaN,NaN,NaN


In [ ]:
!cp drive/"My Drive"/"Dutch Tweets"/*.geojson .

In [ ]:
!pip install geopandas
!pip install geopy


     |████████████████████████████████| 972kB 3.4MB/s 
     |████████████████████████████████| 10.9MB 14.9MB/s 
     |████████████████████████████████| 14.8MB 329kB/s 


In [ ]:
import geopandas as gpd

provinces = gpd.read_file('provinces.geojson')

print(provinces.shape)
provinces.head()

(12, 4)


,name,regioFacetId,level,geometry
0,Drenthe,tcm:106-353397-1024,3,"POLYGON ((6.41328 52.98552, 6.36252 53.03397, ..."
1,Flevoland,tcm:106-353410-1024,3,"POLYGON ((5.36115 52.67573, 5.37726 52.76481, ..."
2,Friesland (Fryslân),tcm:106-353417-1024,3,"POLYGON ((5.08707 53.32307, 5.10178 53.36803, ..."
3,Gelderland,tcm:106-353445-1024,3,"POLYGON ((5.60602 51.99416, 5.59200 52.00139, ..."
4,Groningen,tcm:106-353502-1024,3,"POLYGON ((6.28698 53.34138, 6.27368 53.34527, ..."


In [ ]:
from shapely.geometry import Point, Polygon

In [ ]:
def checkProvince(p1):
  #print(len(p1))
  if p1 != None and len(p1) >= 2 :
    p1 = Point(p1[1], p1[0])  
    provinces['InProvince'] = provinces['geometry'].apply(lambda x: x.contains(p1))
    selProvince = provinces[provinces['InProvince'] == True]
    #print(selProvince)
    if selProvince.shape[0] > 0:
      return selProvince['name'].values[0]
    else:
      return False
  else:
    return False

In [ ]:
allLocation_chunk['Province'] = allLocation_chunk['point'].apply(lambda x: checkProvince(x))
allLocation_chunk.head(15)

,Unnamed: 0,location,point_info,point,latitude,longitude,altitude,Province
0,9171,"Elim, PA","(Elim, Upper Yoder Township, Cambria County, P...","(40.2987335, -78.94353268168663, 0.0)",40.298733,-78.943533,0.0,False
1,9172,@VlkEke,None,None,NaN,NaN,NaN,False
2,9173,Staatsbosbeheer Noord Limburg,None,None,NaN,NaN,NaN,False
3,9174,Zennewijnen,"(Zennewijnen, Tiel, Gelderland, Nederland, (51...","(51.8587463, 5.4118529, 0.0)",51.858746,5.411853,0.0,Gelderland
4,9175,IJsselmuiden,"(IJsselmuiden, Kampen, Overijssel, Nederland, ...","(52.56396385, 5.940171304145334, 0.0)",52.563964,5.940171,0.0,Overijssel
5,9176,"Lima, Perú","(Lima, Peru, (-12.0621065, -77.0365256))","(-12.0621065, -77.0365256, 0.0)",-12.062107,-77.036526,0.0,False
6,9177,Knoal,"(Knoal Hutte, Obergailer Talweg, Obergail, Les...","(46.6517372, 12.790946938592818, 0.0)",46.651737,12.790947,0.0,False
7,9178,Haarlem -A'dam M'tricht Aalt'n,None,None,NaN,NaN,NaN,False
8,9179,"Macragge, Ultramar sector",None,None,NaN,NaN,NaN,False
9,9180,Zeeland/Sud de France,None,None,NaN,NaN,NaN,False


In [ ]:
allLocation_chunk.to_csv("allLocation_chunk"+str(idx)+".csv", index=False)
print("Geo processing completed....allLocation_chunk"+str(idx) + ".csv")

Geo processing completed....allLocation_chunk6.csv


In [ ]:
!cp allLocation_chunk6.csv drive/"My Drive"/"Dutch Tweets"/allLocation_chunk6.csv

In [ ]:
!wget http://davis.wpi.edu/~xmdv/datasets/ohsumed.tar.gz


--2020-12-11 07:17:52--  http://davis.wpi.edu/~xmdv/datasets/ohsumed.tar.gz
Resolving davis.wpi.edu (davis.wpi.edu)... 130.215.28.55
Connecting to davis.wpi.edu (davis.wpi.edu)|130.215.28.55|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6695770 (6.4M) [application/x-gzip]
Saving to: ‘ohsumed.tar.gz’

ohsumed.tar.gz      100%[===================>]   6.38M  1.81MB/s    in 3.5s    

2020-12-11 07:17:55 (1.81 MB/s) - ‘ohsumed.tar.gz’ saved [6695770/6695770]



In [ ]:
!tar -xvzf ohsumed.tar.gz

ohsumed.okc
